Import libraries

In [13]:
import networkx as nx
import torch
import torch_geometric as tg
from torch_geometric.utils import negative_sampling
import torchinfo
import torch_geometric.utils.convert as tgc
import numpy as np
from typing import final
import plotly.graph_objects as go
import random
from models.layers import GATConvBlock, SAGEConvBlock, GCN2ConvBlock, GCNConvBlock
from models.pretraining.encoders import SimpleGCNEncoder, ResGCN2ConvEncoder, RevSAGEConvEncoder, RevGATConvEncoder
from models.pretraining.gae import GAEv2
from models.pretraining.vgae import VGAEv2, VGEncoder

Define graph and plot it

In [14]:
_POSITION_ATTRIBUTE: final = "pos"
_X_MIN: final = 0
_X_MAX: final = 2
_Y_MIN: final = 0
_Y_MAX: final = 2
g = nx.Graph()
g.add_node(0, x=[1., 0., 1.2, 1.1, 0.2, 0.1])
g.add_node(1, x=[0., 1., 0, 1.2, 1.1, 0.2])
g.add_node(2, x=[0.4, 1., 0.1, 0.2, 0.7, 0.3])
g.add_node(3, x=[1., 1.2, 0.9, 0.9, 0.5, 0.4])
g.add_node(4, x=[1., 1.3, 0.4, 0.3, 1.8, 0.45])
g.add_edge(1, 0, edge_weight=1.0)
g.add_edge(1, 2, edge_weight=2.)
g.add_edge(2, 0, edge_weight=1.)
g.add_edge(3, 2, edge_weight=1.)
g.add_edge(4, 2, edge_weight=1.)

print(g.nodes(data=True))
print(g.edges(data=True))
print("ciao")

[(0, {'x': [1.0, 0.0, 1.2, 1.1, 0.2, 0.1]}), (1, {'x': [0.0, 1.0, 0, 1.2, 1.1, 0.2]}), (2, {'x': [0.4, 1.0, 0.1, 0.2, 0.7, 0.3]}), (3, {'x': [1.0, 1.2, 0.9, 0.9, 0.5, 0.4]}), (4, {'x': [1.0, 1.3, 0.4, 0.3, 1.8, 0.45]})]
[(0, 1, {'edge_weight': 1.0}), (0, 2, {'edge_weight': 1.0}), (1, 2, {'edge_weight': 2.0}), (2, 3, {'edge_weight': 1.0}), (2, 4, {'edge_weight': 1.0})]
ciao


Plot graph

In [15]:
if len(nx.get_node_attributes(g, "pos",)) == 0:
    pos = {i: (random.gauss(_X_MIN, _X_MAX), random.gauss(_Y_MIN, _Y_MAX)) for i in g.nodes}
    nx.set_node_attributes(g, pos, "pos")

edge_x = []
edge_y = []
for edge in g.edges():
    x0, y0 = g.nodes[edge[0]]['pos']
    x1, y1 = g.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in g.nodes():
    x, y = g.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(g.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(f'node {node}, # of connections: ' + str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

# noinspection PyTypeChecker
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='<br>Network graph made with Python',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

Convert graph in PyTorch geometric

In [16]:
pyg = tgc.from_networkx(g)
print(pyg)
print(pyg.edge_weight)

Data(x=[5, 6], edge_index=[2, 10], pos=[5, 2], edge_weight=[10])
tensor([1., 1., 1., 2., 1., 2., 1., 1., 1., 1.])


Instantiate layers

In [17]:
gat0 = GATConvBlock(6, 3, dropout=0.3, heads=2, edge_dim=1)
gat1 = GATConvBlock(6, 3, heads=1, edge_dim=1, concat=True)

sage0 = SAGEConvBlock(6, 3, project=True)
sage1 = SAGEConvBlock(6, 3, project=False)

gcn0 = GCNConvBlock(6, 3, normalize=False)
gcn1 = GCNConvBlock(6, 3, normalize=True)

gcn20 = GCN2ConvBlock(6, 0.6)
gcn21 = GCN2ConvBlock(6, 0.5)

print(gat0)
print(gat1)
print(sage0)
print(sage1)
print(gcn0)
print(gcn1)
print(gcn20)
print(gcn21)

print(gat0(pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print(gat1(pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print(sage0(pyg.x, edge_index=pyg.edge_index))
print(sage1(pyg.x, edge_index=pyg.edge_index))
print(gcn0(pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print(gcn1(pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print(gcn20(pyg.x, x0=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print(gcn21(pyg.x, x0=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))

GATConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): GATv2Conv(6, 3, heads=2)
)
GATConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): GATv2Conv(6, 3, heads=1)
)
SAGEConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): SAGEConv(6, 3, aggr=mean)
)
SAGEConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): SAGEConv(6, 3, aggr=mean)
)
GCNConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): GCNConv(6, 3)
)
GCNConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): GCNConv(6, 3)
)
GCN2ConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): GCN2Conv(6, alpha=0.6, beta=1.0)
)
GCN2ConvBlock(
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
  (conv): GCN2Conv(6, alpha=0.5, beta=1.0)
)
tensor([[ 0.5131, -0.3270, -0.0742],
        [-0.0083, -0.3630,  0.0352],
        [ 0.4540, -0.3

Instantiate encoders

In [18]:
gat_enc = RevGATConvEncoder(
    in_channels=6,
    hidden_channels=4,
    out_channels=3,
    num_convs=3,
    dropout=0.0,
    version="v2",
    edge_dim=1,
    heads=8,
    num_groups=2,
    concat=False,
    normalize_hidden=True
)
print("Reversible residual GAT Encoder")
print(gat_enc)
print(torchinfo.summary(gat_enc))
print(gat_enc(pyg.x, pyg.edge_index, pyg.edge_weight))
print("--------------------------------------------------------------------\n\n\n")


sage_enc = RevSAGEConvEncoder(
    in_channels=6,
    hidden_channels=4,
    out_channels=3,
    num_convs=4,
    dropout=0.0,
    project=True,
    root_weight=True,
    num_groups=2,
    aggr='mean',
    normalize_hidden=True
)
print("Reversible residual SAGE Encoder")
print(sage_enc)
print(torchinfo.summary(sage_enc))
print(sage_enc(pyg.x, pyg.edge_index))
print("--------------------------------------------------------------------\n\n\n")


gcn_enc = SimpleGCNEncoder(
    in_channels=6,
    hidden_channels=5,
    out_channels=3,
    conv_dims=[5, 5, 4, 4],
    dropout=0.0,
    improved=True
)
print("Simple GCN Encoder")
print(gcn_enc)
print(torchinfo.summary(gcn_enc))
print(gcn_enc(pyg.x, pyg.edge_index, pyg.edge_weight))
print("--------------------------------------------------------------------\n\n\n")


gcn2_enc = ResGCN2ConvEncoder(
    in_channels=6,
    hidden_channels=5,
    out_channels=3,
    alpha=0.3,
    num_convs=4,
    dropout=0.0
)
print("ResGCN2 Encoder")
print(gcn2_enc)
print(torchinfo.summary(gcn2_enc))
print(gcn2_enc(pyg.x, pyg.edge_index))
print("--------------------------------------------------------------------\n\n\n")

Reversible residual GAT Encoder
RevGATConvEncoder(
  (lin1): Linear(in_features=6, out_features=4, bias=True)
  (lin2): Linear(in_features=4, out_features=3, bias=True)
  (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
  (convs): ModuleList(
    (0): GroupAddRev(GATConvBlock(
      (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
      (conv): GATv2Conv(2, 2, heads=8)
    ), num_groups=2)
    (1): GroupAddRev(GATConvBlock(
      (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
      (conv): GATv2Conv(2, 2, heads=8)
    ), num_groups=2)
    (2): GroupAddRev(GATConvBlock(
      (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
      (conv): GATv2Conv(2, 2, heads=8)
    ), num_groups=2)
  )
)
Layer (type:depth-idx)                                  Param #
RevGATConvEncoder                                       --
├─Linear: 1-1                                           28
├─Linear: 1-2                                           15
├─LayerNorm: 1-3

Test encoders serialization

In [19]:
print("Reversible residual GAT Encoder")
constr_params = gat_enc.serialize_constructor_params()
state_dict = gat_enc.state_dict()
print(constr_params)
gat_enc2 = RevGATConvEncoder.from_constructor_params(constr_params)
gat_enc2.load_state_dict(state_dict)
print(gat_enc2)
print(torchinfo.summary(gat_enc2))
print("\n\nOriginal: ")
print(gat_enc(pyg.x, pyg.edge_index, pyg.edge_weight))
print("\n\nDeserialized: ")
print(gat_enc2(pyg.x, pyg.edge_index, pyg.edge_weight))
print("--------------------------------------------------------------------\n\n\n")


print("Reversible residual SAGE Encoder")
constr_params = sage_enc.serialize_constructor_params()
state_dict = sage_enc.state_dict()
print(constr_params)
sage_enc2 = RevSAGEConvEncoder.from_constructor_params(constr_params)
sage_enc2.load_state_dict(state_dict)
print(sage_enc2)
print(torchinfo.summary(sage_enc2))
print("\n\nOriginal: ")
print(sage_enc(pyg.x, pyg.edge_index))
print("\n\nDeserialized: ")
print(sage_enc2(pyg.x, pyg.edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Simple GCN Encoder")
constr_params = gcn_enc.serialize_constructor_params()
state_dict = gcn_enc.state_dict()
print(constr_params)
gcn_enc2 = SimpleGCNEncoder.from_constructor_params(constr_params)
gcn_enc2.load_state_dict(state_dict)
print(gcn_enc2)
print(torchinfo.summary(gcn_enc2))
print("\n\nOriginal: ")
print(gcn_enc(pyg.x, pyg.edge_index, pyg.edge_weight))
print("\n\nDeserialized: ")
print(gcn_enc2(pyg.x, pyg.edge_index, pyg.edge_weight))
print("--------------------------------------------------------------------\n\n\n")


print("Residual GCN2 Encoder")
constr_params = gcn2_enc.serialize_constructor_params()
state_dict = gcn2_enc.state_dict()
print(constr_params)
gcn2_enc2 = ResGCN2ConvEncoder.from_constructor_params(constr_params)
gcn2_enc2.load_state_dict(state_dict)
print(gcn2_enc2)
print(torchinfo.summary(gcn2_enc2))
print("\n\nOriginal: ")
print(gcn2_enc(pyg.x, pyg.edge_index, pyg.edge_weight))
print("\n\nDeserialized: ")
print(gcn2_enc2(pyg.x, pyg.edge_index, pyg.edge_weight))
print("--------------------------------------------------------------------\n\n\n")

Reversible residual GAT Encoder
{'in_channels': 6, 'hidden_channels': 4, 'out_channels': 3, 'num_convs': 3, 'dropout': 0.0, 'version': 'v2', 'edge_dim': 1, 'heads': 8, 'concat': False, 'num_groups': 2, 'normalize_hidden': True}
RevGATConvEncoder(
  (lin1): Linear(in_features=6, out_features=4, bias=True)
  (lin2): Linear(in_features=4, out_features=3, bias=True)
  (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
  (convs): ModuleList(
    (0): GroupAddRev(GATConvBlock(
      (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
      (conv): GATv2Conv(2, 2, heads=8)
    ), num_groups=2)
    (1): GroupAddRev(GATConvBlock(
      (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
      (conv): GATv2Conv(2, 2, heads=8)
    ), num_groups=2)
    (2): GroupAddRev(GATConvBlock(
      (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
      (conv): GATv2Conv(2, 2, heads=8)
    ), num_groups=2)
  )
)
Layer (type:depth-idx)                                  Param

Instantiate and test GAEv2

In [20]:
print("Reversible residual GAT GAE")
gae = GAEv2(encoder=gat_enc)
print(gae)
print(torchinfo.summary(gae))
print("Reconstruction forward()")
print(gae(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward_all()")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Latent space encoding")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(z)
print("Reconstruction decode()")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(gae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Reversible residual SAGE GAE")
gae = GAEv2(encoder=sage_enc)
print(gae)
print(torchinfo.summary(gae))
print("Reconstruction forward()")
print(gae(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward_all()")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index))
print("Latent space encoding")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(z)
print("Reconstruction decode()")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(gae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Simple GCN GAE")
gae = GAEv2(encoder=gcn_enc)
print(gae)
print(torchinfo.summary(gae))
print("Reconstruction forward()")
print(gae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all()")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Reconstruction decode()")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(gae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Residual GCN2 GAE")
gae = GAEv2(encoder=gcn2_enc)
print(gae)
print(torchinfo.summary(gae))
print("Reconstruction forward()")
print(gae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all()")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Reconstruction decode()")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(gae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")

Reversible residual GAT GAE
GAEv2(
  (encoder): RevGATConvEncoder(
    (lin1): Linear(in_features=6, out_features=4, bias=True)
    (lin2): Linear(in_features=4, out_features=3, bias=True)
    (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
    (convs): ModuleList(
      (0): GroupAddRev(GATConvBlock(
        (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (conv): GATv2Conv(2, 2, heads=8)
      ), num_groups=2)
      (1): GroupAddRev(GATConvBlock(
        (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (conv): GATv2Conv(2, 2, heads=8)
      ), num_groups=2)
      (2): GroupAddRev(GATConvBlock(
        (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (conv): GATv2Conv(2, 2, heads=8)
      ), num_groups=2)
    )
  )
  (decoder): InnerProductDecoder()
)
Layer (type:depth-idx)                                       Param #
GAEv2                                                        --
├─RevGATConvEncoder: 1-1           

Test serialization for GAE

In [21]:
print("Reversible residual GAT GAE")
gae = GAEv2(encoder=gat_enc)
constr_params = gae.serialize_constructor_params()
state_dict = gae.state_dict()
print("Constructor params: ")
print(constr_params)
gae2 = GAEv2.from_constructor_params(constr_params, RevGATConvEncoder)
gae2.load_state_dict(state_dict)
print(gae2)
print(torchinfo.summary(gae2))
print("Reconstruction forward() original")
print(gae(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward() deserialized")
print(gae2(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward_all() original")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward_all() deserialized")
print(gae2.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Latent space encoding original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(z)
print("Latent space encoding deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(z)
print("Reconstruction decode() original")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized")
print(gae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
print(gae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized")
print(gae2.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Reversible residual SAGE GAE")
gae = GAEv2(encoder=sage_enc)
constr_params = gae.serialize_constructor_params()
state_dict = gae.state_dict()
print("Constructor params: ")
print(constr_params)
gae2 = GAEv2.from_constructor_params(constr_params, RevSAGEConvEncoder)
gae2.load_state_dict(state_dict)
print(gae2)
print(torchinfo.summary(gae2))
print("Reconstruction forward() original")
print(gae(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward() deserialized")
print(gae2(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward_all() original")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward_all() deserialized")
print(gae2.forward_all(x=pyg.x, edge_index=pyg.edge_index))
print("Latent space encoding original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(z)
print("Latent space encoding deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index)
print(z)
print("Reconstruction decode() original")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized")
print(gae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
print(gae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized")
print(gae2.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Simple GCN GAE")
gae = GAEv2(encoder=gcn_enc)
constr_params = gae.serialize_constructor_params()
state_dict = gae.state_dict()
print("Constructor params: ")
print(constr_params)
gae2 = GAEv2.from_constructor_params(constr_params, SimpleGCNEncoder)
gae2.load_state_dict(state_dict)
print(gae2)
print(torchinfo.summary(gae2))
print("Reconstruction forward() original")
print(gae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward() deserialized")
print(gae2(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() original")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() deserialized")
print(gae2.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Latent space encoding deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Reconstruction decode() original")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized")
print(gae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
print(gae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized")
print(gae2.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Residual GCN2 GAE")
gae = GAEv2(encoder=gcn2_enc)
constr_params = gae.serialize_constructor_params()
state_dict = gae.state_dict()
print("Constructor params: ")
print(constr_params)
gae2 = GAEv2.from_constructor_params(constr_params, ResGCN2ConvEncoder)
gae2.load_state_dict(state_dict)
print(gae2)
print(torchinfo.summary(gae2))
print("Reconstruction forward() original")
print(gae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward() deserialized")
print(gae2(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() original")
print(gae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() deserialized")
print(gae2.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Latent space encoding deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Reconstruction decode() original")
print(gae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized")
print(gae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
print(gae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized")
print(gae2.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test original")
z = gae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized")
z = gae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(gae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")

Reversible residual GAT GAE
Constructor params: 
{'encoder': {'state_dict': OrderedDict([('lin1.weight', tensor([[ 0.3017,  0.0780,  0.3780, -0.2731, -0.1171,  0.2941],
        [-0.1848,  0.2964, -0.2858, -0.0216, -0.3562,  0.3056],
        [-0.2245,  0.2626, -0.1170,  0.1236, -0.3463,  0.1923],
        [-0.3265, -0.1130, -0.0322, -0.3334, -0.2413, -0.0834]])), ('lin1.bias', tensor([ 0.1149, -0.2528, -0.2822,  0.1887])), ('lin2.weight', tensor([[-0.4647, -0.2695,  0.0881, -0.0825],
        [ 0.1337, -0.1715,  0.0899,  0.0364],
        [ 0.4216, -0.3230, -0.0765,  0.4799]])), ('lin2.bias', tensor([-0.4023, -0.3164, -0.4138])), ('norm.weight', tensor([1., 1., 1., 1.])), ('norm.bias', tensor([0., 0., 0., 0.])), ('convs.0.convs.0.norm.weight', tensor([1., 1.])), ('convs.0.convs.0.norm.bias', tensor([0., 0.])), ('convs.0.convs.0.conv.att', tensor([[[-0.0849,  0.3083],
         [-0.5168,  0.1050],
         [-0.2693,  0.2103],
         [-0.4785, -0.6156],
         [ 0.0797,  0.0698],
        

Instantiate VGAEv2 and test it

In [22]:
print("Reversible residual GAT VGAE")
vgae_enc = VGEncoder(encoder_mu=gat_enc)
vgae = VGAEv2(encoder=vgae_enc)
print(vgae)
print(torchinfo.summary(vgae))
print("Reconstruction forward()")
print(vgae(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward_all()")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Latent space encoding")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(z)
print("Mu")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(mu)
print("log(std)")
print(logstd)
print("Reconstruction decode()")
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(vgae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Reversible residual SAGE VGAE")
vgae_enc = VGEncoder(encoder_mu=sage_enc)
vgae = VGAEv2(encoder=vgae_enc)
print(vgae)
print(torchinfo.summary(vgae))
print("Reconstruction forward()")
print(vgae(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward_all()")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index))
print("Latent space encoding")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(z)
print("Mu")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index)
print(mu)
print("log(std)")
print(logstd)
print("Reconstruction decode()")
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(vgae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Simple GCN VGAE")
vgae_enc = VGEncoder(encoder_mu=gcn_enc)
vgae = VGAEv2(encoder=vgae_enc)
print(vgae)
print(torchinfo.summary(vgae))
print("Reconstruction forward()")
print(vgae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all()")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Mu")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(mu)
print("log(std)")
print(logstd)
print("Reconstruction decode()")
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(vgae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Residual GCN2 VGAE")
vgae_enc = VGEncoder(encoder_mu=gcn2_enc)
vgae = VGAEv2(encoder=vgae_enc)
print(vgae)
print(torchinfo.summary(vgae))
print("Reconstruction forward()")
print(vgae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all()")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Mu")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(mu)
print("log(std)")
print(logstd)
print("Reconstruction decode()")
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction loss")
print(vgae.recon_loss(z, pyg.edge_index))
print("AUC and precision metric test")
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")

Reversible residual GAT VGAE
VGAEv2(
  (encoder): VGEncoder(
    (_encoder_mu): RevGATConvEncoder(
      (lin1): Linear(in_features=6, out_features=4, bias=True)
      (lin2): Linear(in_features=4, out_features=3, bias=True)
      (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
      (convs): ModuleList(
        (0): GroupAddRev(GATConvBlock(
          (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
          (conv): GATv2Conv(2, 2, heads=8)
        ), num_groups=2)
        (1): GroupAddRev(GATConvBlock(
          (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
          (conv): GATv2Conv(2, 2, heads=8)
        ), num_groups=2)
        (2): GroupAddRev(GATConvBlock(
          (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
          (conv): GATv2Conv(2, 2, heads=8)
        ), num_groups=2)
      )
    )
    (_encoder_logstd): RevGATConvEncoder(
      (lin1): Linear(in_features=6, out_features=4, bias=True)
      (lin2): Linear(in_features

Test VGAE serialization

In [23]:
print("Reversible residual GAT VGAE")
vgae_enc = VGEncoder(encoder_mu=gat_enc)
vgae = VGAEv2(encoder=vgae_enc)
print("Constructor params: ")
constr_params = vgae.serialize_constructor_params()
state_dict = vgae.state_dict()
print(constr_params)
vgae2 = VGAEv2.from_constructor_params(constr_params, VGEncoder, RevGATConvEncoder)
vgae2.load_state_dict(state_dict)
print(vgae2)
print(torchinfo.summary(vgae2))
print("forward() original")
print(vgae(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("forward() deserialized (should be ok if they are different because of the randomization)")
print(vgae2(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward_all() original")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Reconstruction forward_all() deserialized (should be ok if they are different because of the randomization)")
print(vgae2.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight))
print("Latent space encoding original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(z)
print("Latent space encoding deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(z)
print("Mu original")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(mu)
print("log(std) original")
print(logstd)
print("Mu deserialized (should be equal to original)")
mu, logstd = vgae2.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(mu)
print("log(std) deserialized (should be equal to original)")
print(logstd)
print("Reconstruction decode() original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(vgae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(vgae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(vgae2.recon_loss(z, pyg.edge_index))
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print("AUC and precision metric test original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_attr=pyg.edge_weight)
print(vgae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Reversible residual SAGE VGAE")
vgae_enc = VGEncoder(encoder_mu=sage_enc)
vgae = VGAEv2(encoder=vgae_enc)
print("Constructor params: ")
constr_params = vgae.serialize_constructor_params()
state_dict = vgae.state_dict()
print(constr_params)
vgae2 = VGAEv2.from_constructor_params(constr_params, VGEncoder, RevSAGEConvEncoder)
vgae2.load_state_dict(state_dict)
print(vgae2)
print(torchinfo.summary(vgae2))
print("forward() original")
print(vgae(x=pyg.x, edge_index=pyg.edge_index))
print("forward() deserialized (should be ok if they are different because of the randomization)")
print(vgae2(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward_all() original")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index))
print("Reconstruction forward_all() deserialized (should be ok if they are different because of the randomization)")
print(vgae2.forward_all(x=pyg.x, edge_index=pyg.edge_index))
print("Latent space encoding original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(z)
print("Latent space encoding deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index)
print(z)
print("Mu original")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index)
print(mu)
print("log(std) original")
print(logstd)
print("Mu deserialized (should be equal to original)")
mu, logstd = vgae2.encoder(x=pyg.x, edge_index=pyg.edge_index)
print(mu)
print("log(std) deserialized (should be equal to original)")
print(logstd)
print("Reconstruction decode() original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index)
print(vgae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(vgae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index)
print(vgae2.recon_loss(z, pyg.edge_index))
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print("AUC and precision metric test original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index)
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index)
print(vgae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Simple GCN VGAE")
vgae_enc = VGEncoder(encoder_mu=gcn_enc)
vgae = VGAEv2(encoder=vgae_enc)
print("Constructor params: ")
constr_params = vgae.serialize_constructor_params()
state_dict = vgae.state_dict()
print(constr_params)
vgae2 = VGAEv2.from_constructor_params(constr_params, VGEncoder, SimpleGCNEncoder)
vgae2.load_state_dict(state_dict)
print(vgae2)
print(torchinfo.summary(vgae2))
print("forward() original")
print(vgae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("forward() deserialized (should be ok if they are different because of the randomization)")
print(vgae2(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() original")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() deserialized (should be ok if they are different because of the randomization)")
print(vgae2.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Latent space encoding deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Mu original")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(mu)
print("log(std) original")
print(logstd)
print("Mu deserialized (should be equal to original)")
mu, logstd = vgae2.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(mu)
print("log(std) deserialized (should be equal to original)")
print(logstd)
print("Reconstruction decode() original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae2.recon_loss(z, pyg.edge_index))
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print("AUC and precision metric test original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")


print("Residual GCN2 VGAE")
vgae_enc = VGEncoder(encoder_mu=gcn2_enc)
vgae = VGAEv2(encoder=vgae_enc)
print("Constructor params: ")
constr_params = vgae.serialize_constructor_params()
state_dict = vgae.state_dict()
print(constr_params)
vgae2 = VGAEv2.from_constructor_params(constr_params, VGEncoder, ResGCN2ConvEncoder)
vgae2.load_state_dict(state_dict)
print(vgae2)
print(torchinfo.summary(vgae2))
print("forward() original")
print(vgae(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("forward() deserialized (should be ok if they are different because of the randomization)")
print(vgae2(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() original")
print(vgae.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Reconstruction forward_all() deserialized (should be ok if they are different because of the randomization)")
print(vgae2.forward_all(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight))
print("Latent space encoding original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Latent space encoding deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(z)
print("Mu original")
mu, logstd = vgae.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(mu)
print("log(std) original")
print(logstd)
print("Mu deserialized (should be equal to original)")
mu, logstd = vgae2.encoder(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(mu)
print("log(std) deserialized (should be equal to original)")
print(logstd)
print("Reconstruction decode() original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae.decode(z, pyg.edge_index))
print("Reconstruction decode() deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae2.decode(z, pyg.edge_index))
print("Reconstruction loss original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae.recon_loss(z, pyg.edge_index))
print("Reconstruction loss deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae2.recon_loss(z, pyg.edge_index))
neg_edge_index = negative_sampling(pyg.edge_index, z.size(0))
print("AUC and precision metric test original")
z = vgae.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("AUC and precision metric test deserialized (should be ok if they are different because of the randomization)")
z = vgae2.encode(x=pyg.x, edge_index=pyg.edge_index, edge_weight=pyg.edge_weight)
print(vgae2.test(z, pyg.edge_index, neg_edge_index=neg_edge_index))
print("--------------------------------------------------------------------\n\n\n")

Reversible residual GAT VGAE
Constructor params: 
{'encoder': {'state_dict': OrderedDict([('_encoder_mu.lin1.weight', tensor([[-0.3068,  0.0589,  0.0485, -0.3847, -0.2539, -0.0323],
        [-0.1433,  0.2027, -0.2680, -0.1653,  0.2488,  0.3773],
        [ 0.1245,  0.2250, -0.1455, -0.2873, -0.3446,  0.1250],
        [ 0.3235,  0.3735, -0.2686,  0.3273,  0.1007, -0.0134]])), ('_encoder_mu.lin1.bias', tensor([-0.2244, -0.3042, -0.2610,  0.2541])), ('_encoder_mu.lin2.weight', tensor([[ 0.0941, -0.4091,  0.2512, -0.2293],
        [ 0.0881, -0.3363, -0.0651,  0.4542],
        [ 0.0251,  0.4587,  0.3038, -0.3096]])), ('_encoder_mu.lin2.bias', tensor([ 0.2598, -0.2703,  0.4144])), ('_encoder_mu.norm.weight', tensor([1., 1., 1., 1.])), ('_encoder_mu.norm.bias', tensor([0., 0., 0., 0.])), ('_encoder_mu.convs.0.convs.0.norm.weight', tensor([1., 1.])), ('_encoder_mu.convs.0.convs.0.norm.bias', tensor([0., 0.])), ('_encoder_mu.convs.0.convs.0.conv.att', tensor([[[-0.2768, -0.6540],
         [ 0.50